In [1]:
import pandas as pd

import numpy as np

import os

os.getcwd()



'/Users/wenzhiwang/Desktop/2018/DataScience/Capstone/src'

In [2]:
# read play, download, search
df_play = pd.read_csv('/Users/wenzhiwang/Desktop/2018/DataScience/Capstone/data/play_ds.csv')

df_play.head()



/Applications/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,uid,device,song_id,date,play_time,song_length
0,168551247,ar,1.18814e+07,2017-03-30,78,149.0
1,168549788,ip,295469,2017-03-30,16,242.0
2,168550496,ar,1.24954e+07,2017-03-30,369,2747.0
3,168544926,ar,4.40379e+06,2017-03-30,6,28.0
4,168532580,ar,2.17629e+07,2017-03-30,67,137.0


In [3]:
df_download = pd.read_csv('/Users/wenzhiwang/Desktop/2018/DataScience/Capstone/data/down_ds.csv')

# initialization download frequency as 1
df_download['download'] = 1

In [4]:
df_download.drop(['device','date'],axis = 1, inplace = True)

# groupby download at uid and song_id level
df_down = df_download.groupby(['uid','song_id']).agg('sum')

df_down.reset_index(inplace = True)

# filter out records with negative play time

df_play['play_time'] = pd.to_numeric(df_play.play_time, errors='coerce')

df_play = df_play[ (df_play.play_time >= 0) & ( df_play.song_length >0) ]

df_play.dropna(inplace = True)

df_play.drop(['device','date'],axis = 1,inplace = True)

df_play_u_song = df_play.groupby(['uid','song_id']).agg('sum')

df_play_u_song.reset_index(inplace = True)

# compute fraction of play time as total song length
df_play_u_song['play'] = df_play_u_song.play_time/df_play_u_song.song_length
# delete crazy record
df_play_u_song = df_play_u_song[df_play_u_song.play <= 1]


# Merge download and play dataset
df = pd.merge(df_play_u_song,df_down, on = ['uid','song_id'], how = 'outer')

df.drop(['play_time','song_length'], axis = 1, inplace = True)

df.head()

# Compute implicit review score based on download behaviour and play behaviour
# The more downloads, the higher the score
# The more play, the higher the score

df['score'] = 0


def Rating(df):
    
    if df[0] >= 5:
        return 7
    
    elif df[0] < 5 and df[0] >= 1 and df[1] > 0 :
        return 6
    
    elif df[0] < 5 and df[0] >= 1 and np.isnan(df[1]):
        return 5
    
    elif np.isnan(df[0]) and df[1] >= 0.8:
        return 4
    
    elif np.isnan(df[0]) and df[1] >= 0.6 and df[1] < 0.8 :
        return 3
    
    elif np.isnan(df[0]) and df[1] >= 0.3 and df[1] < 0.6 :
        return 2
    
    else:
        return 1    
# rename download and play to ger around of unicode issues
df.rename(columns = {'download':0, 'play': 1}, inplace = True)



In [5]:
df['score'] = df[[0,1]].apply(Rating,axis = 1)


In [6]:
df.drop([0,1], axis = 1, inplace = True)

In [7]:
df.head()

,uid,song_id,score
0,250691,5.83485e+06,3
1,250691,6.95632e+06,4
2,250691,7.08174e+06,4
3,250691,9.86321e+06,4
4,250691,9.94339e+06,4


In [8]:
## downsample the whole dataset

df_final = df[np.random.random(df.shape) < 0.001]

df_final.head()

# construct utility matrix

df_utility = pd.pivot_table(data=df_final, 
                            values='score', 
                            index='uid', 
                            columns='song_id', 
                            fill_value=0)

In [9]:
df_utility.head()

song_id,0.0,5252.0,54279.0,54394.0,54588.0,54751.0,54916.0,54993.0,55219.0,55510.0,...,99590,9961549,9962546,996642,996649,99703,99747,997703,998097,999020
uid,,,,,,,,,,,,,,,,,,,,,
2446575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3152758,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6653266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10126063,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11038021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
from time import time

# Item-Item Similarity Matrix
item_sim_mat = cosine_similarity(df_utility.T)

least_to_most_sim_indexes = np.argsort(item_sim_mat, axis=1)

# Neighborhoods
neighborhood_size = 75
neighborhoods = least_to_most_sim_indexes[:, -neighborhood_size:]

# Let's pick a lucky user
user_id = df_utility.index[100]

n_users = df_utility.shape[0]
n_items = df_utility.shape[1]

start_time = time()

items_rated_by_this_user = df_utility.loc[user_id].nonzero()[0]



In [11]:
# Just initializing so we have somewhere to put rating preds
out = np.zeros(n_items)
for item_to_rate in range(n_items):
    relevant_items = np.intersect1d(neighborhoods[item_to_rate],
                                    items_rated_by_this_user,
                                    assume_unique=True)  # assume_unique speeds up intersection op

    out[item_to_rate] = np.dot(df_utility.loc[user_id][list(relevant_items)].values ,\
        item_sim_mat[item_to_rate, list(relevant_items)] )/ \
        item_sim_mat[item_to_rate, list(relevant_items)].sum()


pred_ratings = np.nan_to_num(out)

print(pred_ratings)

print("Execution time: %f seconds" % (time()-start_time))

/Applications/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


[ 0.  0.  0. ...,  0.  0.  0.]
Execution time: 7.457760 seconds


In [12]:
# Recommend n business
n = 10

# Get item indexes sorted by predicted rating
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))[::-1]

# Find items that have been rated by user
items_rated_by_this_user = df_utility.loc[user_id].nonzero()[0]

# We want to exclude the items that have been rated by user
unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                if item not in items_rated_by_this_user]

unrated_items_by_pred_rating[:n]

[8816, 2936, 2941, 2940, 2939, 2938, 2937, 2935, 2934, 2933]

In [13]:
## Another recoomendation: NFM 

ratings_mat = np.mat(df_utility.values)



from sklearn.decomposition import NMF

def fit_nmf(M,k):
    nmf = NMF(n_components=k)
    nmf.fit(M)
    W = nmf.transform(M);
    H = nmf.components_;
    err = nmf.reconstruction_err_
    return W,H,err

# decompose
W,H,err = fit_nmf(ratings_mat,200)

print(err)

print(W.shape,H.shape)





309.615930262
((7862, 200), (200, 8817))


In [14]:
# reconstruct
ratings_mat_fitted = W.dot(H)
errs = np.array((ratings_mat-ratings_mat_fitted).flatten()).squeeze()
mask = np.array(ratings_mat.flatten()).squeeze()>0

mse = np.mean(errs[mask]**2)
average_abs_err = abs(errs[mask]).mean()
print(mse)
print(average_abs_err)


8.6797886392
2.41421002402


In [15]:
user_id = 100
n = 10

pred_ratings = ratings_mat_fitted[user_id,:]
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))[::-1]

items_rated_by_this_user = ratings_mat[user_id].nonzero()[1]

unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                if item not in items_rated_by_this_user]

unrated_items_by_pred_rating[:n]



[8816, 2935, 2941, 2940, 2939, 2938, 2937, 2936, 2934, 2960]

In [16]:
### check errors
# truth
ratings_true = ratings_mat[user_id, items_rated_by_this_user]
# prediction
ratings_pred = pred_ratings[items_rated_by_this_user]
print(list(zip(np.array(ratings_true).squeeze(),ratings_pred)))


err_one_user = ratings_true-ratings_pred


print(err_one_user)


print(abs(err_one_user).mean())

[(1, 0.0), (1, 0.0), (1, 0.0)]
[[ 1.  1.  1.]]
1.0


In [17]:
######## Let's try UVD

from sklearn.decomposition import TruncatedSVD

def fit_uvd(M,k):
    # use TruncatedSVD to realize UVD
    svd = TruncatedSVD(n_components=k, n_iter=7, random_state=0)
    svd.fit(M)

    V = svd.components_
    U = svd.transform(M) # effectively, it's doing: U = M.dot(V.T)
    # we can ignore svd.singular_values_ for our purpose
    
    # why we can do this?
    # recall: 
    # SVD start from u*s*v=M => u*s=M*v.T, where M*v.T is our transformation above 
    # to get U in UVD
    # so the above U is effectively u*s in SVD
    # that's why U*V = u*s*v = M our original matrix
    # there are many ways to understand it!
    # here we by-passed singular values.
    
    return U,V, svd

# decompose
U,V,svd = fit_uvd(ratings_mat,200)


print(U.shape,V.shape)



((7862, 200), (200, 8817))


In [18]:
# reconstruct
ratings_mat_fitted = U.dot(V) # U*V


# recall: U = M.dot(V.T), then this is M.dot(V.T).dot(V)
# original M is transformed to new space, then transformed back
# this is another way to understand it!

# calculate errs
errs = np.array((ratings_mat-ratings_mat_fitted).flatten()).squeeze()
mask = np.array((ratings_mat).flatten()).squeeze()>0

mse = np.mean(errs[mask]**2)
average_abs_err = abs(errs[mask]).mean()
print(mse)
print(average_abs_err)

8.55390955387
2.41102168426


In [19]:
# compare with another way to reconstruct matrix
# with the above "tranformed to the new space and back" language
# without the UV language, we can do:

# reconstruct M with inverse_transform
ratings_mat_fitted_2 = svd.inverse_transform(svd.transform(ratings_mat))
ratings_mat_fitted = U.dot(V)
print(sum(sum(ratings_mat_fitted - ratings_mat_fitted_2)))
# they are just equivalent!!

0.0


In [20]:
# get recommendations for one user
user_id = 100
n = 10

pred_ratings = ratings_mat_fitted[user_id,:]
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))[::-1]

items_rated_by_this_user = ratings_mat[user_id].nonzero()[1]

unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                if item not in items_rated_by_this_user]

unrated_items_by_pred_rating[:n]




[2479, 4499, 5154, 973, 1588, 776, 4613, 3557, 5344, 3787]

In [21]:
### check errors
# truth
ratings_true = ratings_mat[user_id, items_rated_by_this_user]
# prediction
ratings_pred = pred_ratings[items_rated_by_this_user]
print(list(zip(np.array(ratings_true).squeeze(),ratings_pred)))



err_one_user = ratings_true-ratings_pred
print(err_one_user)

print(abs(err_one_user).mean())

[(1, 1.254648206364032e-21), (1, 1.254648206364032e-21), (1, 1.254648206364032e-21)]
[[ 1.  1.  1.]]
1.0
